# Interacting with GPIO from MicroBlaze

In [ ]:
from pynq.overlays.base import BaseOverlay
import time
import multiprocessing
import threading
import os
import socket

base = BaseOverlay("base.bit")

In [ ]:
%%microblaze base.PMODA

#include "gpio.h"
#include "pyprintf.h"

//Function to turn on/off a selected pin of PMODA
void write_gpio(unsigned int pin, unsigned int val){
    if (val > 1){
        pyprintf("pin value must be 0 or 1");
    }
    gpio pin_out = gpio_open(pin);
    gpio_set_direction(pin_out, GPIO_OUT);
    gpio_write(pin_out, val);
}

//Function to read the value of a selected pin of PMODA
unsigned int read_gpio(unsigned int pin){
    gpio pin_in = gpio_open(pin);
    gpio_set_direction(pin_in, GPIO_IN);
    return gpio_read(pin_in);
}

//Add a C++ function to reset all the GPIO pins on the chosen PMOD
void reset_gpio(){
    for (unsigned int j = 0; j <= 7; j = j + 1) { 
        write_gpio(j, 0);
        //write_gpio(pin,val)
        //we want the value reset to zero
        //12 pins, starts at 0
    }
}

In [ ]:
#LED_Status = [0, 0, 2, 3] #default status
LED_Status = [1, 0, 2, 2] #testing status
#          = [off/on, solid/blink, cycle/blue/red/green, brightness_level]
Duty_Cycle = [0.04736614270344993, 0.217637640824031, 0.5310492251033824, 1.0] #25%, 50%, 75%, and 100% brightness.
#time_on = duty_cycle * (1/frequency) && time_off = (1-duty_cycle) * (1/frequency)
frequency = 20 #for the PWM Brightness
PWM_on = []
for i in range(4):
    PWM_on.append(Duty_Cycle[i]/frequency)
PWM_off = []
for i in range(4):
    PWM_off.append((1-Duty_Cycle[i])/frequency)
sleep_time = 1
PWM_run_time = frequency*sleep_time
user_input = False
change_value = 0
change_element = 0
running = True
threads = []

def Change_LED(new_input, element):
    global LED_Status, user_input
    reset_gpio()
    LED_Status[element] = new_input
    user_input = False
    #print(LED_Status)
    
def LED():
    global LED_Status, PWM_on, PWM_off, PWM_run_time, sleep_time
    if LED_Status[0] == 0: #IF OFF
        time.sleep(sleep_time)
    else: #IF ON
        if LED_Status[3] == 3: #full bright (NO PWM)
            if LED_Status[2] == 0: #If NO PWM & Cycle Color
                for i in range(1,4): #cycle write from 1 (blue), 2 (red) and 3 (green)
                    write_gpio(i,1) #turn on  LED
                    time.sleep(sleep_time) #sleep with ON
                    write_gpio(i,0) #turn off LED
                    if LED_Status[1] == 1: #if supposed to blink, sleep
                        time.sleep(sleep_time)
            else: #IF NO PWM & Solid Color
                write_gpio(LED_Status[2],1)
                time.sleep(sleep_time)
                if LED_Status[1] == 1: #if supposed to blink, sleep
                        write_gpio(LED_Status[2],0)
                        time.sleep(sleep_time)
        else: #If NOT full brightness (PWM needed)
            if LED_Status[2] == 0: #PWM & cycle thru colors
                for i in range(1,4):
                    for j in range(PWM_run_time):
                        write_gpio(i,1) #turn on LED
                        time.sleep(PWM_on[LED_Status[3]]) #sleep with ON
                        write_gpio(i,0) #turn off LED
                        time.sleep(PWM_off[LED_Status[3]]) #sleep with OFF
                    if LED_Status[1] == 1: #If blinking
                        time.sleep(sleep_time)
            else:
                for i in range(PWM_run_time):
                    write_gpio(LED_Status[2],1) #turn on LED
                    time.sleep(PWM_on[LED_Status[3]]) #sleep with ON
                    write_gpio(LED_Status[2],0) #turn off LED
                    time.sleep(PWM_off[LED_Status[3]]) #sleep with OFF
                if LED_Status[1] == 1: #If Blinking
                    time.sleep(sleep_time)

def LOOP(_l, num):
    global user_input, Change_LED, change_value, change_element, running
    while running:
        resource_available = _l.acquire(False)
        if resource_available == True: #Note: We always want the same "worker" to have the lock or the LED may not work properly
            LED()
            if user_input == True:
                Change_LED(change_value, change_element)
            _l.release()
            #time.sleep(sleep_time-.1)
            #user_input(1)
        else:
            #INSERT FUNCTION TO SEE IF NEW MESSAGE CAME IN
            #This Function should set user_input = True
            print('Worker {} is searching for incoming message.'.format(num))
            time.sleep(sleep_time)

def Start_RGB():
    global threads
    reset_gpio()            

    fork = threading.Lock()
    for i in range(2):
        #fork is the shared lock
        #i is the user (led number too)
        t = threading.Thread(target=LOOP, args=(fork, i))
        threads.append(t) #creates the new thread number
        t.start()
        
def Stop_RGB():
    global running, threads
    running = False
    for i in range(2):
        threads[i].join()

In [ ]:
def run_pynq_server_R():
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.bind(('192.168.2.99', 12345))
    sock.listen()
    conn, addr = sock.accept()
    with conn:
        print("before pynq R")
        while True:
            print("inside pynq R")
            time.sleep(0.0001)
            data = conn.recv(1024)
            if (data.decode() == 'RGB'):
                print("rgb pynq R")
                Start_RGB()
                break
        while True:
            time.sleep(0.0001)
            data = conn.recv(1024)
            if (data.decode() == 'disconnect'):
                print("disconnect pynq R")
                Stop_RGB()
                break
    print("after pynq R")

def run_pynq_client_R():
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    while True:
        time.sleep(0.0001)
        if base.buttons[0].read() != 0:
            sock.connect(('192.168.2.1', 54321))
            break
    while True: # TODO: replace with numpad logic
        time.sleep(0.0001)
        if base.buttons[1].read() != 0:
            sock.sendall(b'RGB')
            break
    while True:
        time.sleep(0.0001)
        if base.buttons[3].read() != 0:
            sock.sendall(b'disconnect')
            break
    sock.close()

p_server = multiprocessing.Process(target=run_pynq_server_R)
p_server.start()
os.system("taskset -p -c {} {}".format(0, p_server.pid))

p_client = multiprocessing.Process(target=run_pynq_client_R)
p_client.start()
os.system("taskset -p -c {} {}".format(1, p_client.pid))

p_server.join();
p_client.join();